Χρησιμοποιώ τυχαίες εικόνες μαζί με τις δερματολογικές εικόνες της βάσης PAD, ώστε να φτίαξω ένα δυαδικό ταξινομητή, που θα διαχωρίζει τις δερματολογικές εικόνες (in distribution) από τις άσχετες, τυχαίες εικόνες(out of distribution). Αυτός ο ταξινομητής θα χρησιμοποιηθεί στην εφαρμογή ώστε να ελέγχει κατά πόσο είναι έγκυρη η εικόνα που έδωσε ο χρήστης για έλεγχο.

In [ ]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile, os, urllib.request, glob, math, shutil, sys, random
import torchvision
import torch.optim as optim
from torchvision import datasets, models, transforms
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, precision_recall_fscore_support, roc_auc_score
import sklearn.metrics
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import PIL
import PIL.Image

sys.path.append('drive/MyDrive/ood_classification')
import downloader_ood

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Mounted at /content/drive


Link από τα δύο ακόλουθα csv: https://storage.googleapis.com/openimages/web/download_v7.html

In [ ]:
img_annotations = pd.read_csv('drive/MyDrive/ood_classification/oidv7-val-annotations-human-imagelabels.csv')
img_annotations.head()

,ImageID,Source,LabelName,Confidence
0,0001eeaf4aed83f9,verification,/m/013c1m,0.0
1,0001eeaf4aed83f9,verification,/m/013sr7,0.0
2,0001eeaf4aed83f9,verification,/m/015y8h,1.0
3,0001eeaf4aed83f9,verification,/m/015z8s,0.0
4,0001eeaf4aed83f9,verification,/m/015z_b,0.0


In [ ]:
img_data = pd.read_csv('drive/MyDrive/ood_classification/validation-images-with-rotation.csv')
img_data.head()

,ImageID,Subset,OriginalURL,OriginalLandingURL,License,AuthorProfileURL,Author,Title,OriginalSize,OriginalMD5,Thumbnail300KURL,Rotation
0,fe600639ac5f36c1,validation,https://farm2.staticflickr.com/5612/1534025949...,https://www.flickr.com/photos/118815643@N04/15...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/118815643@N04/,LabHacker CD,_GUT6674,242145,0jBpbNION09+r02xkTIBcA==,https://c8.staticflickr.com/6/5612/15340259497...,NaN
1,ba82c70cc6cdf449,validation,https://farm4.staticflickr.com/3859/1527670200...,https://www.flickr.com/photos/125612851@N04/15...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/125612851@N04/,Stockholms stadsarkiv,SE_SSA_0870_Ofört_029,1658910,qMzF2UrrpPdjO+Nhhl6pvQ==,https://c7.staticflickr.com/4/3859/15276702002...,0.0
2,e3ffa4c868b11b15,validation,https://farm6.staticflickr.com/5336/1717037798...,https://www.flickr.com/photos/101125222@N02/17...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/101125222@N02/,Ismaïl Taha,ct26,1085575,MXOLnofbdq410OH34woSeg==,https://c3.staticflickr.com/6/5336/17170377986...,0.0
3,7d00af2927a57eeb,validation,https://c4.staticflickr.com/7/6181/6085189538_...,https://www.flickr.com/photos/66833578@N02/608...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/66833578@N02/,Crycks,Hummingbird,126933,m0dy5a3Pjhbzx15MbScpxg==,https://c5.staticflickr.com/7/6181/6085189538_...,0.0
4,914dd6fb5eb17e85,validation,https://farm6.staticflickr.com/3324/4621899397...,https://www.flickr.com/photos/damo1977/4621899397,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/damo1977/,damo1977,"Christmas 1994 Hoticulture, Nuts & Dried Fruit...",855508,xeHPBJGWTrDVbOufALMFGw==,https://c6.staticflickr.com/4/3324/4621899397_...,NaN


In [ ]:
img_annotations = img_annotations.loc[img_annotations['Confidence'] == 1.0]
img_annotations.drop_duplicates(['ImageID'], inplace = True)
len(img_annotations)

41422

In [ ]:
img_data = img_data.loc[img_data['Rotation'] == 0]
img_data.drop_duplicates(['ImageID'], inplace = True)
len(img_data)

<ipython-input-15-2550d5501af3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  img_data.drop_duplicates(['ImageID'], inplace = True)


35826

In [ ]:
img_dataset = pd.merge(img_data,img_annotations, on = 'ImageID')
img_dataset.head()

,ImageID,Subset,OriginalURL,OriginalLandingURL,License,AuthorProfileURL,Author,Title,OriginalSize,OriginalMD5,Thumbnail300KURL,Rotation,Source,LabelName,Confidence
0,ba82c70cc6cdf449,validation,https://farm4.staticflickr.com/3859/1527670200...,https://www.flickr.com/photos/125612851@N04/15...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/125612851@N04/,Stockholms stadsarkiv,SE_SSA_0870_Ofört_029,1658910,qMzF2UrrpPdjO+Nhhl6pvQ==,https://c7.staticflickr.com/4/3859/15276702002...,0.0,verification,/m/015bv3,1.0
1,e3ffa4c868b11b15,validation,https://farm6.staticflickr.com/5336/1717037798...,https://www.flickr.com/photos/101125222@N02/17...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/101125222@N02/,Ismaïl Taha,ct26,1085575,MXOLnofbdq410OH34woSeg==,https://c3.staticflickr.com/6/5336/17170377986...,0.0,verification,/m/01lcwm,1.0
2,7d00af2927a57eeb,validation,https://c4.staticflickr.com/7/6181/6085189538_...,https://www.flickr.com/photos/66833578@N02/608...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/66833578@N02/,Crycks,Hummingbird,126933,m0dy5a3Pjhbzx15MbScpxg==,https://c5.staticflickr.com/7/6181/6085189538_...,0.0,verification,/m/01280g,1.0
3,eb59a7a5d5518d31,validation,https://farm2.staticflickr.com/4137/4821997585...,https://www.flickr.com/photos/laurenprofeta/48...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/laurenprofeta/,Lauren Mitchell,Kitsune,11163861,LBBlpF5qJhDKfn25/QnKsA==,https://c2.staticflickr.com/5/4137/4821997585_...,0.0,verification,/m/01j0x,1.0
4,4ea213d7e78c2d41,validation,https://farm2.staticflickr.com/2636/3960392021...,https://www.flickr.com/photos/chpaquette/39603...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/chpaquette/,Christopher Paquette,Salsa Verde,1561297,zu+jzw2DK3ujYORR2o/5gg==,https://c7.staticflickr.com/3/2636/3960392021_...,0.0,verification,/m/01ykh,1.0


In [ ]:
len(img_dataset)

35667

In [ ]:
#write to a file
samples = img_dataset.sample(n = 2298)

img_ids = samples['ImageID'].tolist()

f_ids = open("imgFile.txt", "a")

for i in range(2298):
  f_ids.write('validation/' + img_ids[i] + '\n')

f_ids.close()

Κώδικας python για να κατεβάσει τα επιλεγμένα αρχεία. https://raw.githubusercontent.com/openimages/dataset/master/downloader.py

In [ ]:
!python 'drive/MyDrive/ood_classification/downloader_ood.py' 'imgFile.txt' --download_folder='images' --num_processes=5

In [ ]:
#split: 20%

image_name_list = os.listdir('images')
len_images = len(image_name_list)

random.shuffle(image_name_list)

test_images_len = int(len_images / 5)
train_images_len = len_images - test_images_len
print(train_images_len)
print(test_images_len)

src_path_prefix = 'images'
dest_path_train = "train_ood"
dest_path_test = "test_ood"

os.makedirs(dest_path_train)
os.makedirs(dest_path_test)

for i in image_name_list[:train_images_len]:
  src_path = os.path.join(src_path_prefix, i)
  shutil.copy(src_path, dest_path_train)

for i in image_name_list[train_images_len:]:
  src_path = os.path.join(src_path_prefix, i)
  shutil.copy(src_path, dest_path_test)

1839
459


In [ ]:
!ls test_ood | wc -l

459


In [ ]:
shutil.make_archive('drive/MyDrive/ood_classification/train_ood', 'zip', 'train_ood')
shutil.make_archive('drive/MyDrive/ood_classification/test_ood', 'zip', 'test_ood')

'/content/drive/MyDrive/ood_classification/test_ood.zip'

Τώρα η PAD εικόνες.

In [ ]:
urllib.request.urlretrieve("https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/zr7vgbcyr2-1.zip", "imgs.zip")

with zipfile.ZipFile('imgs.zip', 'r') as zip_ref:
  zip_ref.extractall('imgs_folder')

print("Done with unzipping...")

for i in ['imgs_folder/images/imgs_part_1.zip', 'imgs_folder/images/imgs_part_2.zip', 'imgs_folder/images/imgs_part_3.zip']:
  with zipfile.ZipFile(i, 'r') as zip_ref:
    zip_ref.extractall()

dest_folder = 'images_pad'
os.makedirs(dest_folder)

for src_folder in ['imgs_part_1', 'imgs_part_2', 'imgs_part_3']:
  file_list = os.listdir(src_folder)
  print("Found %d files in folder: %s" %(len(file_list),src_folder))
  for filename in file_list:
    src_path = os.path.join(src_folder,filename)
    shutil.copy(src_path, dest_folder)

!ls images_pad | wc -l

In [ ]:
!ls images_pad | wc -l

2298


Split της PAD.

In [ ]:
#split: 20%

image_name_list = os.listdir('images_pad')
len_images = len(image_name_list)

random.shuffle(image_name_list)

test_images_len = int(len_images / 5)
train_images_len = len_images - test_images_len
print(train_images_len)
print(test_images_len)

src_path_prefix = 'images_pad'
dest_path_train = "train_ind"
dest_path_test = "test_ind"

os.makedirs(dest_path_train)
os.makedirs(dest_path_test)

for i in image_name_list[:train_images_len]:
  src_path = os.path.join(src_path_prefix, i)
  shutil.copy(src_path, dest_path_train)

for i in image_name_list[train_images_len:]:
  src_path = os.path.join(src_path_prefix, i)
  shutil.copy(src_path, dest_path_test)

1839
459


In [ ]:
!ls test_ind | wc -l
!ls train_ind | wc -l

459
1839


In [ ]:
shutil.make_archive('drive/MyDrive/ood_classification/train_ind', 'zip', 'train_ind')
shutil.make_archive('drive/MyDrive/ood_classification/test_ind', 'zip', 'test_ind')

'/content/drive/MyDrive/ood_classification/test_ind.zip'